In [ ]:
import pandas as pd
import plotly.express as px
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import RandomOverSampler, SMOTE, ADASYN, BorderlineSMOTE
from imblearn.under_sampling import RandomUnderSampler
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay, f1_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV
import seaborn as sns

In [ ]:
df = pd.read_parquet('FullDataToDb70.parquet')

In [ ]:
df = df.drop(['aircraftSerNum-1','date'], axis="columns")

In [ ]:
snVars = ['DeltaPressAltitude-1a',
       'DeltaBleedOutTemp-1b','DeltaBleedMonPress-1b', 'MeanBleedOutTemp-1b',
       'MeanBleedMonPress-1b', 'MeanPressAltitude-1a','time_since_maintenance']
nDf = df[snVars]
nDf

,DeltaPressAltitude-1a,DeltaBleedOutTemp-1b,DeltaBleedMonPress-1b,MeanBleedOutTemp-1b,MeanBleedMonPress-1b,MeanPressAltitude-1a,time_since_maintenance
0,27.0,19.250,111.625,204.736609,55.770472,35001.902023,1037950
1,32258.0,46.750,138.500,210.901265,84.130888,14700.678297,2405000
2,32215.0,48.750,124.000,207.049568,100.138864,18408.192598,3544700
3,602.0,23.500,142.375,202.832240,71.176793,34995.797687,4616300
4,32945.0,25.625,142.375,206.352423,74.899656,13905.362082,6418300
...,...,...,...,...,...,...,...
138231,2097.0,14.000,13.750,203.989268,52.391456,39460.626571,2966101200
138232,38122.0,24.250,138.125,206.437927,74.699921,17310.217261,2967588300
138233,36853.0,41.000,123.875,202.096109,82.938608,24549.926673,2969328400
138234,98.0,22.375,133.750,203.914419,53.228401,39005.084119,2970426800


In [ ]:
scaler = StandardScaler()
scaler.fit(nDf)
scaler.fit_transform(nDf)
nDf_norma = scaler.transform(nDf)

In [ ]:
df3 = pd.DataFrame(nDf_norma)

In [ ]:
df2 =['preFail','phaseOfFlight-1']
df4 = df[df2]

In [ ]:
dfN = pd.concat([df4,df3],axis=1)

In [ ]:
dfN = dfN.rename(columns={0:'DeltaPressAltitude-1a',1:'DeltaBleedOutTemp-1b',2:'DeltaBleedMonPress-1b',3:'MeanBleedOutTemp-1b',4:'MeanBleedMonPress-1b',5:'MeanPressAltitude-1a',6:'time_since_maintenance'})

In [ ]:
df_x = dfN.drop('preFail', axis ='columns')

In [ ]:
df_y = dfN['preFail']

In [ ]:
x = df_x
y = df_y
sm = BorderlineSMOTE(random_state=42)
X_res, y_res = sm.fit_resample(x, y)

In [ ]:
dfN_O = pd.concat([X_res,y_res], axis='columns')

In [ ]:
# dfNx = dfN_O.drop(['preFail'], axis='columns')

In [ ]:
# X = dfNx
# Y = dfN_O['preFail']
# x_train, x_test, y_train, y_test = train_test_split(X,Y,
#                                                     test_size = 0.3,
#                                                     random_state = 42)

In [ ]:
# clf = ExtraTreesClassifier(n_estimators=400, random_state=42, min_samples_split =2, min_samples_leaf = 1, max_features = 'sqrt', max_depth = 50, bootstrap = False)
# clf.fit(x_train, y_train)
# y_pred = clf.predict(x_test)

In [ ]:
!pip3 install pycaret[full]

In [ ]:
from pycaret.classification import *

In [ ]:
s=setup(dfN_O, target = 'preFail')

,Description,Value
0,Session id,8828
1,Target,preFail
2,Target type,Binary
3,Original data shape,"(266870, 9)"
4,Transformed data shape,"(266870, 9)"
5,Transformed train set shape,"(186809, 9)"
6,Transformed test set shape,"(80061, 9)"
7,Numeric features,8
8,Preprocess,True
9,Imputation type,simple


In [ ]:
melhor_modelo = compare_models()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
et,Extra Trees Classifier,0.9872,0.9984,0.9908,0.9836,0.9872,0.9743,0.9744,22.7890
rf,Random Forest Classifier,0.9802,0.9979,0.9870,0.9738,0.9803,0.9604,0.9605,65.0210
dt,Decision Tree Classifier,0.9438,0.9438,0.9575,0.9319,0.9445,0.8875,0.8879,3.1650
catboost,CatBoost Classifier,0.9361,0.9844,0.9626,0.9141,0.9377,0.8721,0.8734,55.2460
knn,K Neighbors Classifier,0.9318,0.9750,0.9935,0.8843,0.9357,0.8635,0.8702,4.7430
xgboost,Extreme Gradient Boosting,0.9234,0.9783,0.9548,0.8983,0.9257,0.8467,0.8484,2.3640
lightgbm,Light Gradient Boosting Machine,0.8954,0.9631,0.9405,0.8626,0.8999,0.7907,0.7940,9.7980
gbc,Gradient Boosting Classifier,0.8237,0.9087,0.9355,0.7646,0.8414,0.6474,0.6642,67.4230
ada,Ada Boost Classifier,0.7815,0.8654,0.8861,0.7329,0.8022,0.5630,0.5759,13.9860
lr,Logistic Regression,0.5665,0.6027,0.3978,0.6003,0.4785,0.1330,0.1413,0.3990


Processing:   0%|          | 0/69 [00:00<?, ?it/s]

In [ ]:
melhor_modelo

ExtraTreesClassifier(bootstrap=False, ccp_alpha=0.0, class_weight=None,
                     criterion='gini', max_depth=None, max_features='sqrt',
                     max_leaf_nodes=None, max_samples=None,
                     min_impurity_decrease=0.0, min_samples_leaf=1,
                     min_samples_split=2, min_weight_fraction_leaf=0.0,
                     n_estimators=100, n_jobs=-1, oob_score=False,
                     random_state=8828, verbose=0, warm_start=False)

In [ ]:
evaluate_model(melhor_modelo)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Pipeline Plot', 'pipelin…

In [ ]:
create_api(melhor_modelo,"modelo70")

API successfully created. This function only creates a POST API, it doesn't run it automatically. To run your API, please run this command --> !python modelo70.py


In [ ]:
create_docker('Api')

Writing requirements.txt
Writing Dockerfile
Dockerfile and requirements.txt successfully created.
    To build image you have to run --> !docker image build -f "Dockerfile" -t IMAGE_NAME:IMAGE_TAG .
            


In [ ]:
# # salvar modelo
# import pickle
# # salvar o modelo XGBoost (xgb_model) no arquivo sale_xgboost.pkl
# with open('modelo50.pkl', 'wb') as file:
#     pickle.dump(clf, file)